In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
import seaborn as sns; sns.set()

# pd.get_option("display.max_columns")
# pd.set_option('display.max_columns', 100)

参考資料：<br>
https://run.unl.pt/bitstream/10362/33864/1/TGI0135.pdf<br>

In [2]:
DATA_DIR = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021'
STAGE_1 = True

# シード情報特徴量・qualityの作成

In [3]:
regular_results = pd.read_csv(DATA_DIR+'/MRegularSeasonCompactResults.csv')

In [4]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'         
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    output = pd.concat([df, dfswap]).sort_index().reset_index(drop=True)
    
    return output

regular_results = prepare_data(regular_results)
# regular_results = regular_results.loc[:, ['Season', 'T1_TeamID', 'T2_TeamID', 'location', 'NumOT']]
# regular_results = regular_results.loc[regular_results['Season'] >= 2015]
regular_results

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,20,1328,64,1228,81,N,0
2,1985,25,1354,70,1106,77,A,0
3,1985,25,1106,77,1354,70,H,0
4,1985,25,1112,63,1223,56,H,0
...,...,...,...,...,...,...,...,...
333755,2020,128,1299,86,1224,77,N,0
333756,2020,128,1314,53,1393,81,N,0
333757,2020,128,1393,81,1314,53,N,0
333758,2020,128,1389,56,1233,54,N,0


In [5]:
# convert to str, so the model would treat TeamID them as factors
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(str)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(str)

# make it a binary task
regular_results['win'] = np.where(regular_results['T1_Score']>regular_results['T2_Score'], 1, 0)

def team_quality(season):
    """
    Calculate team quality for each season seperately. 
    Team strength changes from season to season (students playing change!)
    So pooling everything would be bad approach!
    """
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_results.loc[regular_results.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    # extracting parameters from glm
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','beta']
    quality['Season'] = season
    # taking exp due to binomial model being used
    quality['quality'] = np.exp(quality['beta'])
    # only interested in glm parameters with T1_, as T2_ should be mirroring T1_ ones
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

In [6]:
%%time
team_quality = pd.concat([team_quality(1985),
                          team_quality(1986),
                          team_quality(1987),
                          team_quality(1988),
                          team_quality(1989),
                          team_quality(1990),
                          team_quality(1991),
                          team_quality(1992),
                          team_quality(1993),
                          team_quality(1994),
                          team_quality(1995),
                          team_quality(1996),
                          team_quality(1997),
                          team_quality(1998),
                          team_quality(1999),
                          team_quality(2000),
                          team_quality(2001),
                          team_quality(2002),
                          team_quality(2003),
                          team_quality(2004),
                          team_quality(2005),
                          team_quality(2006),
                          team_quality(2007),
                          team_quality(2008),
                          team_quality(2009),
                          team_quality(2010),
                          team_quality(2011),
                          team_quality(2012),
                          team_quality(2013),
                          team_quality(2014),
                          team_quality(2015),
                          team_quality(2016),
                          team_quality(2017),
                          team_quality(2018),
                          team_quality(2019),
                          team_quality(2020)
                          ]).reset_index(drop=True)

team_quality

CPU times: user 14min 50s, sys: 57.7 s, total: 15min 48s
Wall time: 10min 12s


,TeamID,beta,Season,quality
0,1102,4.400939e-15,1985,1.000000
1,1103,2.135582e-01,1985,1.238076
2,1104,4.099354e+00,1985,60.301301
3,1106,-7.325023e-01,1985,0.480705
4,1108,1.477263e+00,1985,4.380937
...,...,...,...,...
11589,1463,2.408706e+00,2020,11.119567
11590,1464,3.094491e-01,2020,1.362674
11591,1465,5.779575e-01,2020,1.782394
11592,1466,-8.966147e-01,2020,0.407948


In [9]:
tourney_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1
2247,2019,146,1277,68,1181,67,N,0
2248,2019,152,1403,61,1277,51,N,0
2249,2019,152,1438,63,1120,62,N,0


In [30]:
# 各シードに配置された時の勝率、パワーランク作成
tourney_results = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')
seeds = pd.read_csv(DATA_DIR+'/MNCAATourneySeeds.csv')

tourney_results = prepare_data(tourney_results)

team_quality_copy = team_quality.copy()

team_quality_T1 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T1.columns = ['T1_TeamID','Season','T1_quality']
team_quality_T2 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T2.columns = ['T2_TeamID','Season','T2_quality']

tourney_results['T1_TeamID'] = tourney_results['T1_TeamID'].astype(int)
tourney_results['T2_TeamID'] = tourney_results['T2_TeamID'].astype(int)
tourney_results = tourney_results.merge(team_quality_T1, on = ['T1_TeamID','Season'], how = 'left')
tourney_results = tourney_results.merge(team_quality_T2, on = ['T2_TeamID','Season'], how = 'left')

tourney_results = tourney_results.loc[tourney_results['DayNum'] >= 136].reset_index(drop=True)
tourney_results = tourney_results.loc[tourney_results['Season'] < 2015].reset_index(drop=True)

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

seeds_T1 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T2 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed','T1_division']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed','T2_division']

tourney_results = tourney_results.merge(seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_results = tourney_results.merge(seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_results['T1_powerrank'] = tourney_results.groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False).astype(int)
tourney_results['T2_powerrank'] = tourney_results.groupby(['Season','T2_division'])['T2_quality'].rank(method='dense', ascending=False).astype(int)

tourney_results['win'] = np.where(tourney_results['T1_Score'] > tourney_results['T2_Score'], 1, 0)

mean_win_ratio = pd.DataFrame({'seed_win_ratio': tourney_results.groupby('T1_seed')['win'].mean(),
                               'powerrank_win_ratio': tourney_results.groupby('T1_powerrank')['win'].mean()})
mean_win_ratio = mean_win_ratio.reset_index().rename(columns={'index':'seed'})
mean_win_ratio

,seed,seed_win_ratio,powerrank_win_ratio
0,1,0.796407,0.788187
1,2,0.712871,0.701058
2,3,0.654762,0.664756
3,4,0.609836,0.578947
4,5,0.523810,0.547170
5,6,0.536965,0.524390
6,7,0.466368,0.512295
7,8,0.422330,0.441860
8,9,0.365079,0.400000
9,10,0.387755,0.371728


In [244]:
# シード情報とチームクオリティ、シード勝率の合体
TourneyCompactResults = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')

A_t1 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "TeamID1"})

A_t2 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "TeamID2"})
A_t1.index = A_t1.index.rename(['Season', 'TeamID'])
A_t2.index = A_t2.index.rename(['Season', 'TeamID'])
A_t2
A = A_t1.join(A_t2, how='outer').reset_index().drop(['TeamID1', 'TeamID2'], axis=1)
del A_t1, A_t2

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

l_seed = A.merge(seeds, on=['Season', 'TeamID'], how='left').drop('Seed', axis=1)
# l_seed = l_seed.drop('location_N', axis=1) # locationがリークさせていた→シンプルにそのトーナメントでの試合数になってしまうから
# del A, seeds
# l_seed['division_rank'] = l_seed.division.replace({'W':4, 'X':3, 'Y':2, 'Z':1})
# l_seed['seed_ratio'] = l_seed['seed'] * l_seed['division_rank']
# l_seed.drop(['seed', 'division', 'division_rank'], inplace=True, axis=1)
q_seed = l_seed.merge(team_quality, on=['Season', 'TeamID'], how='left').drop(['beta'], axis=1)\
        .merge(mean_win_ratio, on='seed', how='left')
q_seed 

,Season,TeamID,seed,division,quality,seed_win_ratio,powerrank_win_ratio
0,1985,1104,7,X,60.301301,0.466368,0.512295
1,1985,1112,10,X,26.511951,0.387755,0.371728
2,1985,1116,9,X,40.730603,0.365079,0.400000
3,1985,1120,11,Z,45.880625,0.358289,0.393939
4,1985,1130,11,Y,46.265708,0.358289,0.393939
...,...,...,...,...,...,...,...
2281,2019,1439,4,W,25.587572,0.609836,0.578947
2282,2019,1449,9,Y,10.468700,0.365079,0.400000
2283,2019,1458,5,Z,26.839622,0.523810,0.547170
2284,2019,1459,7,Y,26.820409,0.466368,0.512295


# シード情報以外で回す場合はここから！

In [290]:
MRSCResults = pd.read_csv(DATA_DIR + '/MRegularSeasonCompactResults.csv')

A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)

H_l = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_H"})
N_l = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_N"})
A_l = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_A"})
lost = A_l.join(N_l, how='outer').join(H_l, how='outer').fillna(0)

win.index = win.index.rename(['Season', 'TeamID'])
lost.index = lost.index.rename(['Season', 'TeamID'])
wl = win.join(lost, how='outer').reset_index()
wl['win_pct_A'] = wl['win_A'] / (wl['win_A'] + wl['lost_A'])
wl['win_pct_N'] = wl['win_N'] / (wl['win_N'] + wl['lost_N'])
wl['win_pct_H'] = wl['win_H'] / (wl['win_H'] + wl['lost_H'])
wl['win_pct_All'] = (wl['win_A'] + wl['win_N'] + wl['win_H']) / \
    (wl['win_A'] + wl['win_N'] + wl['win_H'] + wl['lost_A']\
     + wl['lost_N'] + wl['lost_H'])

del A_w, N_w, H_w, H_l, N_l, A_l, win, lost

In [291]:
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_N", "relScore": "relScore_N"})
H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_H", "relScore": "relScore_H"})
All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\
    .mean().rename(columns={"Score": "Score_All", "relScore": "relScore_All"})
scr = A_scr.join(N_scr, how='outer').join(H_scr, how='outer')\
    .join(All_scr, how='outer').fillna(0).reset_index()

del w_scr, l_scr, wl_scr, A_scr, H_scr, N_scr, All_scr

<ipython-input-291-3cd860bc9e4b>:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
<ipython-input-291-3cd860bc9e4b>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
<ipython-input-291-3cd860bc9e4b>:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
<ipython-input-291-3cd860bc9e4b>:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\


In [292]:
scr

,Season,TeamID,Score_A,relScore_A,Score_N,relScore_N,Score_H,relScore_H,Score_All,relScore_All
0,1985,1102,62.833333,-8.166667,55.50,-12.500000,64.900000,-1.600000,63.083333,-5.791667
1,1985,1103,60.800000,-8.300000,0.00,0.000000,61.230769,1.000000,61.043478,-3.043478
2,1985,1104,65.181818,4.000000,53.00,-0.333333,73.687500,11.937500,68.500000,7.800000
3,1985,1106,72.272727,-7.272727,64.50,-14.000000,74.000000,5.000000,71.625000,-3.791667
4,1985,1108,78.583333,1.583333,77.75,5.250000,91.222222,17.666667,83.000000,7.960000
...,...,...,...,...,...,...,...,...,...,...
11589,2020,1463,72.058824,4.647059,77.00,21.000000,78.666667,11.888889,74.535714,8.142857
11590,2020,1464,69.125000,-6.875000,0.00,0.000000,73.400000,5.733333,71.193548,-0.774194
11591,2020,1465,68.090909,-0.181818,0.00,0.000000,81.812500,8.000000,76.222222,4.666667
11592,2020,1466,69.294118,-7.235294,0.00,0.000000,64.636364,3.090909,67.464286,-3.178571


In [293]:
# ここでチーム毎の試合データをもとに特徴量作成をやっている(1試合毎のやつ)
# 攻撃回数=2P試投数 + 3P試投数 + ターンオーバー数 + FT試投数 × 0.43)を用いて算出した。
# そして、総得点数を攻撃回数で除することで攻撃効率(Points Per Possession)を求めた。

MRSDetailedResults = pd.read_csv(DATA_DIR + '/MRegularSeasonDetailedResults.csv')
# 対戦相手のスコアを保持
MRSDetailedResults['WO_PTS'] = MRSDetailedResults.LScore
MRSDetailedResults['LO_PTS'] = MRSDetailedResults.WScore
# 対戦相手のオフェンスリバウンド回数、ディフェンスリバウンド率を保持
MRSDetailedResults['WO_DREB'] = MRSDetailedResults.LDR
MRSDetailedResults['LO_DREB'] = MRSDetailedResults.WDR
MRSDetailedResults['WO_OREB'] = MRSDetailedResults.LOR
MRSDetailedResults['LO_OREB'] = MRSDetailedResults.WOR
MRSDetailedResults['WO_REB'] = MRSDetailedResults.LDR + MRSDetailedResults.LOR
MRSDetailedResults['LO_REB'] = MRSDetailedResults.WDR + MRSDetailedResults.WOR
#対戦相手のFGA
MRSDetailedResults['WO_FGA'] = MRSDetailedResults.LFGA
MRSDetailedResults['LO_FGA'] = MRSDetailedResults.WFGA
# 対戦相手のFTA
MRSDetailedResults['WO_FTA'] = MRSDetailedResults.LFTA
MRSDetailedResults['LO_FTA'] = MRSDetailedResults.WFTA
# 対戦相手のTO
MRSDetailedResults['WO_TO'] = MRSDetailedResults.LTO
MRSDetailedResults['LO_TO'] = MRSDetailedResults.WTO

w = MRSDetailedResults.loc[:, ['Season', 'WScore', 'WTeamID', 'WFGM','WFGA','WFGM3'
                               ,'WFGA3','WFTM','WFTA','WOR','WDR','WAst',
                               'WTO','WStl','WBlk','WPF', 'WO_PTS', 'WO_DREB', 'WO_OREB', 'WO_REB',
                              'WO_FGA', 'WO_FTA', 'WO_TO']]
w.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']
l = MRSDetailedResults.loc[:, ['Season', 'LScore', 'LTeamID', 'LFGM','LFGA','LFGM3',
                               'LFGA3','LFTM','LFTA','LOR','LDR','LAst',
                               'LTO','LStl','LBlk','LPF', 'LO_PTS', 'LO_DREB', 'LO_OREB', 'LO_REB',
                              'LO_FGA', 'LO_FTA', 'LO_TO']]
l.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']

detail = pd.concat([w,l])
detail['goal_rate'] = detail.FGM / detail.FGA 
detail['3p_goal_rate'] = detail.FGM3 / detail.FGA3  
detail['ft_goal_rate'] = detail.FTM  / detail.FTA 
detail['total_attack'] = detail.FGA + detail.TO + detail.FTA * 0.43
detail['Points_Per_Possession'] = detail.Score / detail.total_attack
# https://bstatsplus.com/glossary このサイト参考にスタッツを計算
# 参考サイトその2:https://www.nogawanogawa.com/entry/basketball_analysis
# POSS:ポゼッション。攻撃回数。
detail['POSS'] = detail.FGA + 0.44 * detail.FTA + detail.TO - detail.OR
# ORTG:オフェンスレーティング。攻撃力。100POSSあたりの得点。
detail['ORTG'] = detail.Score / detail.POSS * 100
# DRTG:ディフェンスレーティング。守備力。100POSSあたりの失点。
detail['DRTG'] = detail.O_PTS / detail.POSS * 100
# NRTG:ネットレーティング。総合力。攻撃力と守備力の差でみる真の力。
detail['NRTG'] = detail.ORTG - detail.DRTG
# eFG%:得点効率。Effective FG%。「Four Factors」の1つ。3Pの価値は2Pの1.5倍という重みを付けて効率を評価。50%を超えるのが目安。
detail['eFG%'] = (detail.FGM + 0.5 * detail.FGM3) / detail.FGA
# PPT:シュート1本あたりの得点。得点期待値？Points Per Shot
detail['FGM2'] = detail.FGM - detail.FGM3 - detail.FTM
detail['PPS'] = (2 * detail.FGM2 + 3 * detail.FGM3) / detail.FGA
# 2PPS:2Pの得点期待値。1を超えるのが目安。
detail['FGA2'] = detail.FGA - detail.FGA3 - detail.FTA
detail['2PPS'] = 2 * detail.FGA2
# 3PPS:3Pの得点期待値。1を超えるのが目安。
detail['3PPS'] = 3 * detail['3p_goal_rate']
# TOV%:ターンオーバー率。「Four Factors」の1つ。1回の攻撃でどれだけTOVするか。少ないほうがよい。
detail['TOV%'] = detail.TO / detail.POSS
# OREB%:オフェンスリバウンド率。「Four Factors」の1つ。OREBの機会にどれだけ奪えるか。OREB機会は自チームのOREBと相手のDREBの合計。
detail['OREB%'] = detail.OR / (detail.OR + detail.O_DREB)
# DREB%:ディフェンスリバウンド率。DREBの機会にどれだけ奪えるか。DREB機会は自チームのDREBと相手のOREBの合計。
detail['DREB%'] = detail.DR / (detail.DR + detail.O_OREB)
# REB%:リバウンド率。リバウンド機会にどれだけ奪えるか。
detail['REB'] = detail.DR + detail.OR
# REB = detail.DR + detail.OR
detail['REB%'] = (detail.REB) / (detail.REB + detail.O_REB)
# FTR:フリースローレーティング。「Four Factors」の1つ。フリースローをどれだけ獲得できるか。アタックできてるかどうかの指標。
detail['FTR'] = detail.FTA / detail.FGA
# TS%:True Shooting Percentage。真のシュート成功率。eFG%がフィールドゴールだけなのに対して、TS%はFTも加味した得点効率。
detail['TS%'] = 0.5 * detail.Score / (detail.FGA + 0.44 * detail.FTA)
# 3P/2P:3PAと2PAの比率。アウトサイドとインサイドのどちらに重きを置いているかを測る指標。
detail['3P/2P'] = detail.FGA3 / detail.FGA2
# AST%:アシスト率。アシストによるフィールドゴールがどれだけあるかを測る指標。
detail['AST%'] = detail.Ast / detail.FGM
# AST/TO:ASTとTOVの比率。アシストが多くターンオーバーが少ないのが優秀とみることが多い。
detail['AST/TO'] = detail.Ast / detail.TO
# PACE:ペース。試合展開の速さの目安。1試合＝40分あたりの攻撃回数。
detail['PACE'] = detail.POSS * 5 / 200
# 以下小川さんの作った特徴量
#対戦相手のPOSS
detail['O_POSS'] = detail.O_FGA + 0.44 * detail.O_FTA + detail.O_TO - detail.O_OREB
#LO_DRTG:対戦相手のディフェンスレーティング
detail['O_DRTG'] = detail.Score / detail.O_POSS * 100
#AdjEM 	自チームのオフェンス　ー　対戦相手のディフェンス
detail['AdjEM'] = detail.ORTG - detail.O_DRTG


dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
                                         'OR','DR','Ast','TO','Stl','Blk','PF',
                                          'goal_rate', '3p_goal_rate',
                                         'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
                                        'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
                                        '2PPS', '3PPS', 'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%', '3P/2P',
                                        'AST%', 'AST/TO', 'PACE', 'AdjEM', 'REB', 'REB%']\
                                        .mean().fillna(0).reset_index()

# dt = detail.groupby(['Season','TeamID'])['Points_Per_Possession', 'NRTG', 'PACE', 'TS%', 'AdjEM',
#                                          'eFG%', 'TOV%', 'OREB%', 'DREB%', 'FTR']\
#                                         .mean().fillna(0).reset_index()

# dt.drop(['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF'], axis=1, inplace=True)
# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%', '3P/2P',
#                                         'AST%', 'AST/TO', 'PACE', '2PPS', '3PPS']\
#                                         .mean().fillna(0).reset_index()

del w, l

<ipython-input-293-5964d49f709d>:96: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',


In [294]:
dt

,Season,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,...,DREB%,FTR,TS%,3P/2P,AST%,AST/TO,PACE,AdjEM,REB,REB%
0,2003,1102,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,...,0.630384,0.446693,0.606248,inf,0.665276,1.375064,1.364071,-0.314082,21.000000,0.410022
1,2003,1103,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,...,0.626998,0.465135,0.585812,1.633822,0.560276,1.324485,1.751963,-0.052350,29.703704,0.468022
2,2003,1104,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,...,0.686897,0.372350,0.521729,1.093940,0.504413,0.992390,1.652536,-0.188675,37.500000,0.526490
3,2003,1105,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,...,0.641434,0.359501,0.504339,0.758505,0.593128,0.861957,1.909538,-1.480413,36.615385,0.479236
4,2003,1106,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,...,0.679342,0.307563,0.509554,0.669325,0.496643,0.723189,1.682000,-0.593199,36.142857,0.514005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6182,2020,1463,26.392857,57.285714,8.892857,24.071429,12.857143,17.892857,8.607143,29.357143,...,0.794472,0.315180,0.572003,2.301997,0.556134,1.215522,1.738786,0.418174,37.964286,0.542331
6183,2020,1464,25.580645,61.548387,7.193548,23.064516,12.838710,18.451613,13.709677,25.032258,...,0.707948,0.306983,0.513489,1.889372,0.469527,1.038393,1.704581,-0.291041,38.741935,0.525781
6184,2020,1465,25.296296,58.925926,9.259259,24.555556,16.370370,20.777778,11.037037,28.481481,...,0.745691,0.361637,0.560249,0.473120,0.491479,1.168315,1.738741,0.524909,39.518519,0.532436
6185,2020,1466,22.892857,54.678571,6.642857,19.821429,15.035714,21.714286,9.785714,25.714286,...,0.740700,0.408001,0.529058,2.190215,0.454017,0.734307,1.737071,-0.247233,35.500000,0.507161


In [295]:
detail

,Season,Score,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,...,REB%,FTR,TS%,3P/2P,AST%,AST/TO,PACE,O_POSS,O_DRTG,AdjEM
0,2003,68,1104,27,58,3,14,11,18,14,...,0.542857,0.310345,0.515777,0.538462,0.481481,0.565217,1.873,70.68,96.208263,-5.444782
1,2003,70,1272,26,62,8,20,10,19,15,...,0.488636,0.306452,0.497442,0.869565,0.615385,1.230769,1.709,67.80,103.244838,-0.845774
2,2003,73,1266,24,58,8,18,17,29,17,...,0.447917,0.500000,0.515828,1.636364,0.625000,1.500000,1.594,64.12,113.849033,0.642811
3,2003,56,1296,18,38,3,9,17,31,6,...,0.403226,0.815789,0.542215,-4.500000,0.611111,0.916667,1.441,57.60,97.222222,-0.067469
4,2003,77,1400,30,61,6,14,11,13,17,...,0.520000,0.213115,0.577038,0.411765,0.400000,0.857143,1.593,62.88,122.455471,-1.614291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92827,2020,54,1233,17,46,2,16,18,20,4,...,0.407895,0.434783,0.492701,1.600000,0.352941,0.352941,1.695,66.44,81.276340,-1.630322
92828,2020,53,1314,20,60,2,16,11,13,13,...,0.518519,0.216667,0.403226,0.516129,0.450000,0.500000,1.768,71.32,74.312956,0.630483
92829,2020,68,1111,20,46,3,15,25,35,8,...,0.446429,0.760870,0.553746,-3.750000,0.300000,0.333333,1.785,72.04,94.392004,0.846091
92830,2020,69,1427,26,67,10,27,7,7,12,...,0.478873,0.104478,0.492295,0.818182,0.423077,0.846154,1.777,69.80,98.853868,-1.780148


In [296]:
# 個人が集計したチームランキング
MMOrdinals = pd.read_csv(DATA_DIR + '/MMasseyOrdinals.csv')

MOR_127_128 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 127) \
                 | (MMOrdinals.RankingDayNum == 128))]\
                [['Season','TeamID','OrdinalRank']]
MOR_50_51 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 50) \
                 | (MMOrdinals.RankingDayNum == 51))]\
                [['Season','TeamID','OrdinalRank']]
MOR_15_16 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 15) \
                 | (MMOrdinals.RankingDayNum == 16))]\
                [['Season','TeamID','OrdinalRank']]

MOR_127_128 = MOR_127_128.rename(columns={'OrdinalRank':'OrdinalRank_127_128'})
MOR_50_51 = MOR_50_51.rename(columns={'OrdinalRank':'OrdinalRank_50_51'})
MOR_15_16 = MOR_15_16.rename(columns={'OrdinalRank':'OrdinalRank_15_16'})

MOR = MOR_127_128.merge(MOR_50_51, how='left', on=['Season','TeamID'])\
        .merge(MOR_15_16, how='left', on=['Season','TeamID'])

## normalizing Rank values by its season maxium as it varies by seasons
MOR_max = MOR.groupby('Season')['OrdinalRank_127_128','OrdinalRank_50_51',
                                'OrdinalRank_15_16'].max().reset_index()
MOR_max.columns = ['Season', 'maxRank_127_128', 'maxRank_50_51', 'maxRank_15_16']

MOR_tmp = MMOrdinals[(MMOrdinals.SystemName == 'MOR') \
                     & (MMOrdinals.RankingDayNum < 133)]
MOR_stats = MOR_tmp.groupby(['Season','TeamID'])['OrdinalRank']\
            .agg(['max','min','std','mean']).reset_index()
MOR_stats.columns = ['Season','TeamID','RankMax','RankMin','RankStd','RankMean']

MOR = MOR.merge(MOR_max, how='left', on='Season')\
        .merge(MOR_stats, how='left', on=['Season','TeamID'])
MOR['OrdinalRank_127_128'] = MOR['OrdinalRank_127_128'] / MOR['maxRank_127_128']
MOR['OrdinalRank_50_51'] = MOR['OrdinalRank_50_51'] / MOR['maxRank_50_51']
MOR['OrdinalRank_15_16'] = MOR['OrdinalRank_15_16'] / MOR['maxRank_15_16']
MOR['RankTrans_50_51_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_50_51']
MOR['RankTrans_15_16_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_15_16']

# MOR['RankMax'] = MOR['RankMax'] / MOR['maxRank_127_128']
# MOR['RankMin'] = MOR['RankMin'] / MOR['maxRank_127_128']
# MOR['RankStd'] = MOR['RankStd'] / MOR['maxRank_127_128']
# MOR['RankMean'] = MOR['RankMean'] / MOR['maxRank_127_128']

MOR.drop(['OrdinalRank_50_51','OrdinalRank_15_16', 'maxRank_50_51'
          ,'maxRank_15_16'],axis=1, inplace=True)

del MOR_127_128, MOR_50_51, MOR_15_16, MOR_max, MOR_tmp, MOR_stats

<ipython-input-296-9e529f19c022>:25: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  MOR_max = MOR.groupby('Season')['OrdinalRank_127_128','OrdinalRank_50_51',


In [297]:
MOR

,Season,TeamID,OrdinalRank_127_128,maxRank_127_128,RankMax,RankMin,RankStd,RankMean,RankTrans_50_51_to_127_128,RankTrans_15_16_to_127_128
0,2003,1102,0.418960,327,146,106,12.670160,127.928571,0.030581,NaN
1,2003,1103,0.431193,327,184,129,15.144723,156.142857,-0.070336,NaN
2,2003,1104,0.058104,327,41,13,8.482691,22.571429,0.012232,NaN
3,2003,1105,0.948012,327,314,262,17.052762,296.785714,0.097859,NaN
4,2003,1106,0.896024,327,293,242,11.841675,268.928571,0.088685,NaN
...,...,...,...,...,...,...,...,...,...,...
6247,2020,1463,0.226629,353,192,26,37.452410,59.722222,0.127479,-0.053824
6248,2020,1464,0.634561,353,278,159,26.999395,235.444444,-0.031161,0.014164
6249,2020,1465,0.674221,353,260,105,46.961986,177.611111,0.365439,-0.005666
6250,2020,1466,0.787535,353,317,224,23.802633,269.277778,-0.011331,-0.065156


In [298]:
wl_1 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_1.columns ]

wl_2 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_2.columns ]

scr_1 = scr.copy()
scr_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_1.columns ]

scr_2 = scr.copy()
scr_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_2.columns ]

dt_1 = dt.copy()
dt_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_1.columns ]

dt_2 = dt.copy()
dt_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_2.columns ]

MOR_1 = MOR.copy()
MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_1.columns ]

MOR_2 = MOR.copy()
MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_2.columns ]

q_seed_1 = q_seed.copy()
q_seed_1.columns = [str(col) + '_1' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_1.columns]

q_seed_2 = q_seed.copy()
q_seed_2.columns = [str(col) + '_2' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_2.columns]

In [299]:
TCResults = pd.read_csv(DATA_DIR + '/MNCAATourneyCompactResults.csv')

tourney1 = TCResults.loc[:, ['Season','WTeamID','LTeamID']]
tourney1.columns = ['Season','TeamID1','TeamID2']
tourney1['result'] = 1

tourney2 = TCResults.loc[:, ['Season','LTeamID','WTeamID']]
tourney2.columns = ['Season','TeamID1','TeamID2']
tourney2['result'] = 0

tourney = pd.concat([tourney1, tourney2])
del tourney1, tourney2

In [300]:
def merge_data(df):

    df = df.merge(wl_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(wl_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)


    df = df.merge(scr_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(scr_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

#     df['win_pct_A_diff'] = df['win_pct_A_1'] - df['win_pct_A_2']
#     df['win_pct_N_diff'] = df['win_pct_N_1'] - df['win_pct_N_2']
#     df['win_pct_H_diff'] = df['win_pct_H_1'] - df['win_pct_H_2']
#     df['win_pct_All_diff'] = df['win_pct_All_1'] - df['win_pct_All_2']

#     df['Score_A_diff'] = df['Score_A_1'] - df['Score_A_2']
#     df['Score_N_diff'] = df['Score_N_1'] - df['Score_N_2']
#     df['Score_H_diff'] = df['Score_H_1'] - df['Score_H_2']
#     df['Score_All_diff'] = df['Score_All_1'] - df['Score_All_2']

#     df['relScore_A_diff'] = df['relScore_A_1'] - df['relScore_A_2']
#     df['relScore_N_diff'] = df['relScore_N_1'] - df['relScore_N_2']
#     df['relScore_H_diff'] = df['relScore_H_1'] - df['relScore_H_2']
#     df['relScore_All_diff'] = df['relScore_All_1'] - df['relScore_All_2']

    df = df.merge(dt_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(dt_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(MOR_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(MOR_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)
    
    df = df.merge(q_seed_1, how='left', left_on=['Season', 'TeamID1'],
                 right_on=['Season', 'TeamID'])
    df = df.merge(q_seed_2, how='left', left_on=['Season', 'TeamID2'],
                 right_on=['Season', 'TeamID'])
    df = df.drop(['TeamID_x', 'TeamID_y'], axis=1)

#     df['OrdinalRank_127_128_diff'] = df['OrdinalRank_127_128_1'] \
#         - df['OrdinalRank_127_128_2']

    df = df.fillna(-1)
    
    for col in df.columns:
        if (df[col] == np.inf).any() or (df[col] == -np.inf).any():
            df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
    
    return df

tourney = merge_data(tourney)
tourney = tourney.loc[tourney.Season >= 1985,:].reset_index(drop=True)

if STAGE_1:
    tourney = tourney.loc[tourney.Season < 2015, :]

<ipython-input-300-74a6617b6398>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
<ipython-input-300-74a6617b6398>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1


In [301]:
if STAGE_1:
    MSampleSubmission = pd.read_csv(DATA_DIR + '/MSampleSubmissionStage1.csv')
else:
    MSampleSubmission = pd.read_csv(DATA_DIR + None) # put stage 2 submission file link here

test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1['TeamID1'] = test1.ID.apply(lambda x: int(x[5:9]))
test1['TeamID2'] = test1.ID.apply(lambda x: int(x[10:14]))

test2 = MSampleSubmission.copy()
test2['Season'] = test2.ID.apply(lambda x: int(x[0:4]))
test2['TeamID1'] = test2.ID.apply(lambda x: int(x[10:14]))
test2['TeamID2'] = test2.ID.apply(lambda x: int(x[5:9]))

test = pd.concat([test1,test2]).drop(['Pred'], axis=1)
test = merge_data(test)

<ipython-input-300-74a6617b6398>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
<ipython-input-300-74a6617b6398>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1


In [302]:
tourney.to_csv('tourney.csv', index=False)
test.to_csv('test.csv', index=False)

In [303]:
tourney

,Season,TeamID1,TeamID2,result,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,...,seed_1,division_1,quality_1,seed_win_ratio_1,powerrank_win_ratio_1,seed_2,division_2,quality_2,seed_win_ratio_2,powerrank_win_ratio_2
0,1985,1116,1234,1,0.333333,0.700000,0.909091,0.636364,0.333333,0.666667,...,9,X,40.730603,0.365079,0.400000,8,X,78.592573,0.422330,0.441860
1,1985,1120,1345,1,0.454545,0.833333,0.666667,0.620690,0.600000,1.000000,...,11,Z,45.880625,0.358289,0.393939,6,Z,103.662745,0.536965,0.524390
2,1985,1207,1250,1,0.857143,1.000000,0.923077,0.925926,0.363636,0.500000,...,1,W,948.524667,0.796407,0.788187,16,W,0.411558,0.000000,0.008264
3,1985,1229,1425,1,0.600000,1.000000,0.785714,0.740741,0.727273,0.666667,...,9,Y,30.425161,0.365079,0.400000,8,Y,31.417978,0.422330,0.441860
4,1985,1242,1325,1,0.600000,0.500000,1.000000,0.766667,0.583333,0.600000,...,3,Z,85.215277,0.654762,0.664756,14,Z,16.885103,0.142857,0.154930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,2014,1277,1163,0,0.700000,0.875000,0.750000,0.764706,0.636364,0.833333,...,4,W,1318.822909,0.609836,0.578947,7,W,932.449371,0.466368,0.512295
4163,2014,1276,1246,0,0.636364,0.714286,0.866667,0.757576,0.500000,0.500000,...,2,Y,1506.324238,0.712871,0.701058,8,Y,732.888339,0.422330,0.441860
4164,2014,1196,1163,0,0.833333,1.000000,1.000000,0.941176,0.636364,0.833333,...,1,X,5308.910097,0.796407,0.788187,7,W,932.449371,0.466368,0.512295
4165,2014,1458,1246,0,0.727273,0.800000,0.823529,0.787879,0.500000,0.500000,...,2,Z,1638.191507,0.712871,0.701058,8,Y,732.888339,0.422330,0.441860


In [304]:
tourney['Season'].unique()

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014])

In [305]:
test

,ID,Season,TeamID1,TeamID2,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,...,seed_1,division_1,quality_1,seed_win_ratio_1,powerrank_win_ratio_1,seed_2,division_2,quality_2,seed_win_ratio_2,powerrank_win_ratio_2
0,2015_1107_1112,2015,1107,1112,0.705882,-1.0,0.800000,0.750000,0.727273,1.000000,...,14,W,17.900901,0.142857,0.154930,2,Z,691.305669,0.712871,0.701058
1,2015_1107_1116,2015,1107,1116,0.705882,-1.0,0.800000,0.750000,0.583333,0.666667,...,14,W,17.900901,0.142857,0.154930,5,Z,199.247783,0.523810,0.547170
2,2015_1107_1124,2015,1107,1124,0.705882,-1.0,0.800000,0.750000,0.545455,0.500000,...,14,W,17.900901,0.142857,0.154930,3,Z,274.108031,0.654762,0.664756
3,2015_1107_1125,2015,1107,1125,0.705882,-1.0,0.800000,0.750000,0.466667,1.000000,...,14,W,17.900901,0.142857,0.154930,15,W,13.870964,0.062500,0.083969
4,2015_1107_1129,2015,1107,1129,0.705882,-1.0,0.800000,0.750000,0.615385,0.666667,...,14,W,17.900901,0.142857,0.154930,11,W,69.981413,0.358289,0.393939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22775,2019_1449_1459,2019,1459,1449,0.785714,1.0,0.923077,0.866667,0.636364,0.571429,...,7,Y,26.820409,0.466368,0.512295,9,Y,10.468700,0.365079,0.400000
22776,2019_1449_1463,2019,1463,1449,0.615385,1.0,0.846154,0.750000,0.636364,0.571429,...,14,W,3.076752,0.142857,0.154930,9,Y,10.468700,0.365079,0.400000
22777,2019_1458_1459,2019,1459,1458,0.785714,1.0,0.923077,0.866667,0.615385,0.600000,...,7,Y,26.820409,0.466368,0.512295,5,Z,26.839622,0.523810,0.547170
22778,2019_1458_1463,2019,1463,1458,0.615385,1.0,0.846154,0.750000,0.615385,0.600000,...,14,W,3.076752,0.142857,0.154930,5,Z,26.839622,0.523810,0.547170
